!jt -t chesterish
# !jt -r

In [9]:
from genetic_algorithm import *
from movement import *
from numpy.random import default_rng
import pickle
import random 
import binascii

import matplotlib.pyplot as plt
# import math
from matplotlib.animation import FuncAnimation
# %matplotlib notebook

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 2
nr_of_genes = 8
nr_individuals = 60

nr_of_generations = 10

## world size
world_size = 10
world_size_x = world_size
world_size_y = world_size

## generations

In [4]:
# for gen_nr in range(nr_of_generations):
#     print(gen_nr)
#     if gen_nr == 0:
#         result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
#         result, a,b,c,d,e = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
#         print(a,b,c,d,e)

time_list = []
result = {}
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        result = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
    else:
        
        result = asexual_reproduction_and_mutation(world_size, result, nr_individuals)
        if result:
            result = next_generation(result,nr_of_input, nr_of_actions, nr_of_inner, world_size, nr_individuals)
            result = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
    result.update(result)
    
    dic_color = {}
    for indiv in result:
        rgb_colors = list(map(hex_to_rgb, result[indiv]['genome']))
        color = tuple(pd.DataFrame(rgb_colors).median()) 
        dic_color[indiv] = color
        
    coords = generate_dictionary_of_coords(result, world_size*2, dic_color)
    
    def update(i):
        ax.clear()
        ax.set_facecolor(plt.cm.Blues(.2))

        ax.set_xlim([0,world_size_x])
        ax.set_ylim([0,world_size_y])
        ax.set_title('moving')
        ax.scatter(x=coords[i]['x'],y=coords[i]['y'], c=coords[i]['color'], s=20, marker='o')
        [spine.set_visible(False) for spine in ax.spines.values()]


    fig, ax = plt.subplots(figsize=(6,6))
    myAnimation = FuncAnimation(
        fig = fig,
        func = update,
        frames = len(coords),
        interval = 10, repeat=False
    )
    myAnimation.save(f'./output/generation-{gen_nr}.gif', writer='imagemagick' , fps=10)
    # create a binary pickle file 
    f = open(f'./output/generation-{gen_nr}.pkl',"wb")
    pickle.dump(result, f)
    f.close()

In [5]:
# def steps_in_generation(world_size, result, world_size_x, world_size_y):
#     for step_nr in range(int(world_size)):
#         for indiv in result:
#             position = result[indiv]['position']
#             x, y = position[-1][0], position[-1][1]
#             if step_nr < 1:
#                 calculate_position(result, indiv, x, y, world_size_x, world_size_y)
#             elif step_nr >= 1:
#                 apply_input(result, indiv)
#                 calculate_position(result, indiv, x, y, world_size_x, world_size_y)

#             # prevent_overlap_movement(last_pos_list, result)
#             for prev in sorted(result.keys()): 
#                 if prev != indiv and result[prev]['position'][-1] == position[-1]:
#                     position[-1] = position[-2]


In [3]:
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        s = time.time()
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
        
        e = time.time()
    else:
        s = time.time()
        result = asexual_reproduction_and_mutation(world_size, result, nr_individuals)
        if result:
            result = next_generation(result,nr_of_input, nr_of_actions, nr_of_inner, world_size, nr_individuals)
            steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
        e = time.time()
    # print(e-s)
    # print(result[0]['position'])

0
1
2
3
4
5
6
7
8
9


In [12]:

out4 = {f'out{str(weight)}': random.random() for weight in range(4)}

In [6]:
result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)

In [28]:
result[0]['out']

{'out0': 0.482, 'out3': 0.983}

In [31]:
out4

{'out0': 0.4796205872250582,
 'out1': 0.37714768548295674,
 'out2': 0.9471701970901344,
 'out3': 0.7749270030660038}

In [33]:
result_1 = {key: out4.get(key, 0) + result[0]['out'].get(key, 0) for key in max([out4,result[0]['out']], key=len)}

In [34]:
result_1

{'out0': 0.9616205872250582,
 'out1': 0.37714768548295674,
 'out2': 0.9471701970901344,
 'out3': 1.757927003066004}

In [30]:
result_1

{'out0': 0, 'out1': 0, 'out2': 0, 'out3': 0}